In [1]:
from bs4 import BeautifulSoup as soup
from urllib.request import urlopen as ureq
import time
import re

# 1. import list of technical elective courses provided by UBC office

In [7]:
import pandas as pd
df = pd.read_excel(r'UBC MechEng Technical Electives Names.xlsx', header = None)
df

,0
0,APSC 461
1,MECH 386
2,MECH 410D
3,MECH 435
4,MECH 436
5,MECH 460
6,MECH 470
7,MECH 473
8,MECH 477
9,MECH 479


In [8]:
course_codes = list(df[0])

In [10]:
#format course codes such that they can be used to search for the courses on course description website
#letter suffixes do not exist in course codes on course description website

course_codes = [code[:8] for code in course_codes]
course_codes = list(dict.fromkeys(course_codes)) #remove duplicates
course_codes

['APSC 461',
 'MECH 386',
 'MECH 410',
 'MECH 435',
 'MECH 436',
 'MECH 460',
 'MECH 470',
 'MECH 473',
 'MECH 477',
 'MECH 479',
 'MECH 481',
 'MECH 485',
 'MECH 488',
 'MECH 491',
 'MECH 493',
 'MECH 502',
 'MECH 433',
 'MECH 445',
 'MECH 462',
 'MECH 464',
 'MECH 468',
 'MECH 469',
 'MECH 478',
 'MECH 484',
 'MECH 489',
 'MECH 495',
 'MECH 496']

In [11]:
len(course_codes)

27

# 2. Use the course codes to find and scrape corresponding course names and descriptions

In [12]:
subject_links = {
    "MECH": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=MECH",
    "APSC": "http://www.calendar.ubc.ca/vancouver/courses.cfm?page=code&institution=2&code=APSC"
}

In [13]:
course_names = []
course_descs = []
counter = 0
prev_link = ""

for course_code in course_codes:
    
    if subject_links[course_code.split()[0]] != prev_link:
        uClient = ureq(subject_links[course_code.split()[0]])
        time.sleep(2)
        page_html = uClient.read()
        uClient.close()
        page_soup = soup(page_html, "html.parser")
        prev_link = subject_links[course_code.split()[0]]
        
    course_title_container = page_soup.find(text = re.compile("^" + course_code))
    if course_title_container is None:
        print("Cannot find ", course_code)
    else:
        course_names.append(course_title_container.findNext().text.strip())
        course_descs.append(course_title_container.findNext().findNext().text.strip())
    
        counter += 1
        print("scraped ", course_code)
    
print(counter)

scraped  APSC 461
scraped  MECH 386
scraped  MECH 410
scraped  MECH 435
scraped  MECH 436
scraped  MECH 460
scraped  MECH 470
scraped  MECH 473
scraped  MECH 477
scraped  MECH 479
scraped  MECH 481
scraped  MECH 485
scraped  MECH 488
scraped  MECH 491
scraped  MECH 493
scraped  MECH 502
scraped  MECH 433
scraped  MECH 445
scraped  MECH 462
scraped  MECH 464
scraped  MECH 468
scraped  MECH 469
scraped  MECH 478
scraped  MECH 484
scraped  MECH 489
scraped  MECH 495
scraped  MECH 496
27


# 3. Inspect collected data and write to CSV

In [14]:
course_names

['Global Engineering Leadership',
 'Industrial Fluid Mechanics',
 'Special Topics in Mechanical Engineering',
 'Orthopaedic Biomechanics',
 'Fundamentals of Injury Biomechanics',
 'Advanced Mechanics of Materials',
 'Energy Conversion Systems',
 'Heating, Ventilating and Air Conditioning',
 'Aerospace Propulsion',
 'Introduction to Computational Fluid Dynamics',
 'Aerodynamics of Aircraft I',
 'Aircraft Design: Structures',
 'Introduction to Ship Hydrodynamics',
 'Computer-Aided Manufacturing',
 'Introduction to Academic Research',
 'Fluid Mechanics',
 'Biofluids',
 'Fuel Cell Systems',
 'Finite Element Analysis',
 'Industrial Robotics',
 'Modern Control Engineering',
 'Dynamic System Modelling',
 'Internal Combustion Engines',
 'Aircraft Design: Aerodynamics',
 'Experimental Thermofluids',
 'Industrial Engineering',
 'Engineering Management']

In [16]:
course_descs

['Introduction to concepts, theory, and practice of engineering leadership, including characteristics; individual and cultural differences, service and management contexts; managing change, conflicts, and crises; real-world ethics and core values. This course is not eligible for Credit/D/Fail grading. [3-0-0] Prerequisite: Completion of third year.',
 'Analysis of piping networks. Review of pumps, turbines and hydraulic motors. Flow measurement devices such as flow meters and transducers for measuring velocity and pressure. Multiphase flows. Introduction to turbulence, mixing and buoyancy driven flows. This course is not eligible for Credit/D/Fail grading. [3-1-0] Prerequisite: MECH 380.',
 'Lectures and readings on specialized topics of current interest in Mechanical Engineering. This course is not eligible for Credit/D/Fail grading.',
 'Musculoskeletal anatomy. Muscle and joint loads. Muscle mechanics. Musculoskeletal dynamics. Gait. Tissue mechanics of tendon, ligament, articular ca

In [17]:
df = pd.DataFrame({
    
    "Course Number": course_codes,
    "Course Name": course_names,
    "Course Description": course_descs    
    
})

df

,Course Number,Course Name,Course Description
0,APSC 461,Global Engineering Leadership,"Introduction to concepts, theory, and practice..."
1,MECH 386,Industrial Fluid Mechanics,"Analysis of piping networks. Review of pumps, ..."
2,MECH 410,Special Topics in Mechanical Engineering,Lectures and readings on specialized topics of...
3,MECH 435,Orthopaedic Biomechanics,Musculoskeletal anatomy. Muscle and joint load...
4,MECH 436,Fundamentals of Injury Biomechanics,Introduction to injury biomechanics. Anatomy. ...
5,MECH 460,Advanced Mechanics of Materials,Axisymmetric membrane stresses in thin shells ...
6,MECH 470,Energy Conversion Systems,Energy supply and demand. Energy conversion ch...
7,MECH 473,"Heating, Ventilating and Air Conditioning",Principles of air conditioning; psychometrics ...
8,MECH 477,Aerospace Propulsion,"Cycle analysis of jet engines, thermodynamic c..."
9,MECH 479,Introduction to Computational Fluid Dynamics,Techniques for numerical solution of ordinary ...


In [18]:
df.to_csv('UBC_MechEng_Technical_Electives_Courses.csv', index = False)